# Simple CNN Model

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
from torchvision.transforms import transforms
from PIL import Image
import numpy as np
import os
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import gc
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
labels=[]
features=[]
for dirname, _, filenames in os.walk('../input/plantdisease/PlantVillage'): #PlantVillage Dataset Path
    for filename in filenames:
        features.append(filename)
        labels.append(dirname)

In [ ]:
def get_sketches():
    TRANSFORM_IMG = transforms.Compose([
        transforms.Resize((225, 225)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    DATA_DIR = "../input/plantdisease/PlantVillage" 
    train_data = datasets.ImageFolder(DATA_DIR, transform=TRANSFORM_IMG)
    test_data = datasets.ImageFolder(DATA_DIR, transform=TRANSFORM_IMG)
    num_train = len(train_data)
    indices = list(range(num_train))
    valid_size = .2
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader

In [ ]:
get_sketches()

In [ ]:
num_epochs = 50
num_classes = 15 
batch_size = 64
learning_rate = 0.001

print(num_epochs, num_classes, batch_size, learning_rate)

In [ ]:
class Sketch_a_Net_CNN(torch.nn.Module):
    def __init__(self):
        super(Sketch_a_Net_CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=15, stride=3, padding=0), #kernel size is filter size
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1 , padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1 , padding=1),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1 , padding=1),
            nn.ReLU()
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1 , padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=7, stride=1 , padding=0),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )


        self.layer7 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=1, stride=1 , padding=0),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(512, num_classes)
        )

         
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return(out)
    

In [ ]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Sketch_a_Net_CNN().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_loader, val_loader = get_sketches()

    total_step = len(train_loader)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(train_loader), 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        torch.save(model.state_dict(), "model.%d" % epoch)

        model.eval()

        train_correct = 0
        train_five_correct = 0
        train_total = 0

        
        with torch.no_grad():
            for data in tqdm(train_loader):
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                _, predicted_five = torch.topk(outputs.data, 5, dim=1)

                train_total += labels.size(0)

                train_correct += (predicted == labels).sum().item()
        
        print('Top One Error of the network on train images: %d %%' % (
                100 * (1 - train_correct / train_total)))

       # print('Top Five Error of the network on train images: %d %%' % (
        #   100 * (1 - train_five_correct /train_total)))

        correct = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(val_loader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                _, predicted_five = torch.topk(outputs.data, 5, dim=1)

                val_total += labels.size(0)

                correct += (predicted == labels).sum().item()
        

        print('Top One Error of the network on validation images: %d %%' % (
                100 * (1 - correct / val_total)))

       # print('Top Five Error of the network on validation images: %d %%' % (
        #   100 * (1 - val_correct / val_total)))
        
        gc.collect()

In [ ]:
train()

In [ ]:
MODEL_PATH = "./model.22" 
TRAINING_IMAGES_DIRECTORY = "../input/plantdisease/PlantVillage"
testing_image = "../input/testingdata/tomato leaf mold.jpg"

In [ ]:
def sketchnet():
    model = Sketch_a_Net_CNN()
    return model
def load_model():
    model = sketchnet()
    model_path = MODEL_PATH 
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint)
    return model
def construct_transformer():
    """construct transformer for images"""
    transformer = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.Resize((225, 225)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return transformer

In [ ]:
def load_categories():
    """load the classification id-name dictionary"""
    classes = os.listdir(TRAINING_IMAGES_DIRECTORY) 
    classes.sort()
    class_to_idx = {i: classes[i] for i in range(len(classes))}
    return class_to_idx

In [ ]:
def main(image_path):
    # load classification categories
    categories = load_categories()
    print(categories)
    # setup the device for running
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # load model and set to evaluation mode

    model = load_model()
    model.to(device)
    model.eval()

    # set image transformer
    transformer = construct_transformer()
    image = Image.open(image_path)
    image = image.convert('RGB')
    
    image = transformer(image)
    image = image.view([1, 3, 225, 225])
    image = image.to(device)


    # run the forward process
    prediction = model(image)
    _, cls = torch.max(prediction, dim=1)

    # output
    index = cls.data.tolist()[0]
    print()
    print(index)

    # Assign PRED_CATEGORY to correct value
    pred_category = categories[index] #"Drew (and not the one from Despicable Me!)"
    print("The predicted category is " + pred_category)

In [ ]:
main(tesing_image)

# VGG-16 Model from scratch

In [ ]:
import pandas as pd
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.models
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
Dataframe=pd.read_csv("../input/dataframe-final/Dataframe_final (1).csv")
train_df, test_df = train_test_split(Dataframe, test_size=0.2)
datagen=ImageDataGenerator(rescale=1./255.)  

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_df,
x_col="Plant disease",
y_col="Plant type",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

valid_generator=datagen.flow_from_dataframe(
dataframe=test_df,
x_col="Plant disease",
y_col="Plant type",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=15, activation="softmax"))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist=model.fit_generator(generator=train_generator,
                    steps_per_epoch=100,
                    validation_data=valid_generator,
                    validation_steps=10,
                    callbacks=[checkpoint,early],
                    epochs=100)

In [ ]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy"])
plt.show()

# Pre-trained VGG-16 Model

In [ ]:
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
vgg=VGG16(include_top=False,weights='imagenet',input_shape=(100,100,3))

for layer in vgg.layers:
   layer.trainable = False

x=Flatten()(vgg.output)
prediction=Dense(15,activation='softmax')(x)
model=tf.keras.Model(inputs=vgg.input,outputs=prediction)

model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
Dataframe=pd.read_csv("../input/dataframeeeeeeeee/Dataframe_final (1).csv")

In [ ]:
train_df, test_df = train_test_split(Dataframe, test_size=0.2)
datagen=ImageDataGenerator(rescale=1./255.)  

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_df,
x_col="Plant disease",
y_col="Plant type",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(100,100))

valid_generator=datagen.flow_from_dataframe(
dataframe=test_df,
x_col="Plant disease",
y_col="Plant type",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(100,100))

In [ ]:
image_files=train_df.shape[0]
valid_files=test_df.shape[0]
batch_size=32
epochs=50

In [ ]:
r = model.fit_generator(
 train_generator,
 validation_data=valid_generator,
 epochs=epochs,
 steps_per_epoch=image_files//batch_size,
 validation_steps=valid_files//batch_size,
)

# Grad-CAM on Pre-trained VGG-16 Model

In [ ]:
from PIL import Image

In [ ]:
image_path = "../input/plantdisease/PlantVillage/Tomato_Early_blight/1742dfdd-c687-46ff-b958-34194feb68e4___RS_Erly.B 7453.JPG"
img = Image.open(image_path)
newsize = (100, 100)
img = img.resize(newsize)
img.save('7.jpg')

In [ ]:
model = keras.models.load_model('../input/vggpretrained-94acc/model (3).h5') #best one

In [ ]:
img_size = (100,100)
decode_predictions = keras.applications.vgg16.decode_predictions
last_conv_layer_name = "block5_conv3"
img_path='./7.jpg'
display(Image(img_path))

In [ ]:
def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))n
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
img_array = get_img_array(img_path, size=img_size)
model=model
model.layers[-1].activation = None
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
plt.matshow(heatmap)
plt.show()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)
    heatmap = np.uint8(255 * heatmap)
    jet = cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    superimposed_img.save(cam_path)
    display(Image(cam_path))

save_and_display_gradcam(img_path, heatmap)